In [ ]:
import numpy as np
import re
from IPython.display import clear_output

from keras.layers import Dense,LSTM,Input,Dropout,Embedding
from keras.utils import np_utils
from keras.models import Model,load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import LambdaCallback

In [ ]:
load_saved_model = False
train_model = False 

In [ ]:
token_type = 'word'

In [ ]:
seq_lenght = 20
filename = './pg19994.txt'
with open(filename, encoding='utf-8') as f:
    text = f.read()

start = text.find("THE FOX AND THE GRAPES\n\n")
end = text.find("ILLUSTRATIONS\n\n\n")
text = text[start:end]

In [ ]:
start_story = '| ' * seq_lenght
text = start_story + text
text = text.lower()
text = text.replace('\n\n\n\n\n',start_story)
text = text.replace('\n',' ')
text = re.sub(' +','. ', text).strip()
text = text.replace('..','.')
text = re.sub('([!"#$%&()*+,-./:;<=>?@[\]^_`{|}~])',r' \1',text)
text = re.sub('\s{2,}',' ', text)

In [ ]:
len(text)

211339

In [ ]:
text

' | . | . | . | . | . | . | . | . | . | . | . | . | . | . | . | . | . | . | . | . the . fox . and . the . grapes . a . fox . one . day . spied . a . beautiful . bunch . of . ripe . grapes . hanging . from . a . vine . trained . along . the . branches . of . a . tree . the . grapes . seemed . ready . to . burst . with . juice , . and . the . fox\'s . mouth . watered . as . he . gazed . longingly . at . them . the . bunch . hung . from . a . high . branch , . and . the . fox . had . to . jump . for . it . the . first . time . he . jumped . he . missed . it . by . a . long . way . so . he . walked . off . a . short . distance . and . took . a . running . leap . at . it , . only . to . fall . short . once . more . again . and . again . he . tried , . but . in . vain . now . he . sat . down . and . looked . at . the . grapes . in . disgust . "what . a . fool . i . am , " . he . said . "here . i . am . wearing . myself . out . to . get . a . bunch . of . sour . grapes . that . are . not . wo

In [ ]:
if token_type == 'word':
    tokenizer = Tokenizer(char_level = False, filters= '')
else:
    tokenizer = Tokenizer(char_level = True, filters= '', lower = False)

tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
token_list = tokenizer.texts_to_sequences([text])[0]

In [ ]:
total_words

3852

In [ ]:
print(tokenizer.word_index)
print(token_list)

{'.': 1, '|': 2, 'the': 3, ',': 4, 'and': 5, 'a': 6, 'to': 7, 'of': 8, 'he': 9, '"': 10, 'in': 11, 'his': 12, 'you': 13, 'was': 14, 'it': 15, 'that': 16, 'with': 17, 'not': 18, 'for': 19, 'had': 20, 'as': 21, 'they': 22, 'at': 23, 'but': 24, '_': 25, 'him': 26, 'is': 27, 'i': 28, 'so': 29, 'one': 30, 'very': 31, 'all': 32, 'on': 33, '!': 34, 'said': 35, 'be': 36, 'fox': 37, 'when': 38, 'out': 39, 'up': 40, '[illustration': 41, ']': 42, 'wolf': 43, 'have': 44, 'from': 45, 'were': 46, 'this': 47, 'or': 48, 'by': 49, 'them': 50, 'gutenberg': 51, 'are': 52, 'who': 53, 'project': 54, 'your': 55, 'ass': 56, 'lion': 57, 'an': 58, 'would': 59, 'their': 60, 'could': 61, 'do': 62, 'if': 63, 'there': 64, 'about': 65, 'me': 66, 'no': 67, '?': 68, 'then': 69, 'into': 70, 'any': 71, 'day': 72, 'much': 73, 'away': 74, 'her': 75, 'work': 76, 'down': 77, 'will': 78, 'saw': 79, '-tm': 80, 'get': 81, 'did': 82, 'time': 83, 'what': 84, 'now': 85, 'dog': 86, 'been': 87, 'can': 88, 'made': 89, 'soon': 90, '

In [ ]:

def generate_sequence(token_list,step):
    x = []
    y = []

    for i in range(0,len(token_list) - seq_lenght, step):
        x.append(token_list[i: i + seq_lenght])
        y.append(token_list[i + seq_lenght])

    y = np_utils.to_categorical(y, num_classes= total_words)
    num_seq = len(x)
    print("Ilość sekwencji: ", num_seq, "\n")

    return x,y,num_seq

step = 1
seq_lenght = 20
x,y,num_seq = generate_sequence(token_list,step)

x = np.array(x)
y = np.array(y)

Ilość sekwencji:  62532 



In [ ]:
x.shape

(62532, 20)

In [ ]:
y.shape

(62532, 3852)

In [ ]:
if load_saved_model:
    
    model = load_model('./saved_models/aesop_dropout_100.h5')

else:
    n_units = 256
    embedding_size = 100

    text_in = Input(shape = (None,))
    embedding = Embedding(total_words,embedding_size)
    x = embedding(text_in)
    x = LSTM(n_units)(x)
    #x = Dropout(0.2)(x)
    text_out = Dense(total_words, activation = 'softmax')(x)
    
    model = Model(text_in,text_out)

    opti = RMSprop(learning_rate = 0.001)
    model.compile(loss='categorical_crossentropy',optimizer=opti)


In [ ]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_4 (Embedding)     (None, None, 100)         385200    
                                                                 
 lstm_3 (LSTM)               (None, 256)               365568    
                                                                 
 dense_3 (Dense)             (None, 3852)              989964    
                                                                 
Total params: 1,740,732
Trainable params: 1,740,732
Non-trainable params: 0
_________________________________________________________________


In [ ]:

def sample_with_temp(preds,temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)/temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1,preds,1)
    return np.argmax(probas)

def generate_text(seed_text,next_words,model,max_sequence_len,temp):
    output_text = seed_text

    seed_text = start_story + seed_text

    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = token_list[-max_sequence_len:]
        token_list = np.reshape(token_list,(1,max_sequence_len))

        probs = model.predict(token_list,verbose=0)[0]
        y_class = sample_with_temp(probs,temperature= temp)

        if y_class == 0:
            output_word = ''
        else:
            output_word = tokenizer.index_word[y_class]
        
        if output_word == "|":
            break
        
        if token_type == 'word':
            output_text += output_word + ' '
            seed_text += output_word + ' '
        else:
            output_text += output_word + ' '
            seed_text += output_word + ' '
        
        return output_text

In [ ]:
def on_epoch_end(epoch,logs):
    seed_text = ""
    gen_word = 500

    print("Temp 0.2")
    print (generate_text(seed_text,gen_words,model,seq_lenght,temp=0.2))
    print("Temp 0.5")
    print (generate_text(seed_text,gen_words,model,seq_lenght,temp=0.5))
    print("Temp 1.0")
    print (generate_text(seed_text,gen_words,model,seq_lenght,temp=1))

if train_model:
    epochs = 1000
    batch_size = 32
    num_batches = int(len(x)/batch_size)
    callback = LambdaCallback(on_epoch_end=on_epoch_end)
    model.fit(
        x,y,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[callback], 
        shuffle=True)

In [ ]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_4 (Embedding)     (None, None, 100)         385200    
                                                                 
 lstm_3 (LSTM)               (None, 256)               365568    
                                                                 
 dense_3 (Dense)             (None, 3852)              989964    
                                                                 
Total params: 1,740,732
Trainable params: 1,740,732
Non-trainable params: 0
_________________________________________________________________


In [ ]:
seed_text = "the frog an the lion ."
gen_words = 500
temp = 0.1
print(generate_text(seed_text, gen_words, model, seq_lenght, temp))

the frog an the lion .beginning 


In [ ]:
def generate_human_led_text(model,max_sequence_len):
    output_text = ' '

    seed_text = start_story

    while 1:
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = token_list[-max_sequence_len:]
        token_list = np.reshape(token_list,(1,max_sequence_len))

        probs = model.predict(token_list,verbose=0)[0]

        top_10_idx = np.flip(np.argsort(probs)[-10:])
        top_10_probs = [probs[x] for x in top_10_idx]
        top_10_words = tokenizer.sequences_to_texts([x] for x in top_10_idx)

        for prob,word in zip(top_10_probs,top_10_words):
            print('{:<6.1%}: {}'.format(prob,word))        

        chosen_word = input()
               
        if chosen_word == "|":
            break
        
        
        output_text += chosen_word + ' '
        seed_text += chosen_word + ' '
        
        clear_output()
         
        
        print(output_text)

In [ ]:
generate_human_led_text(model, 20)

 six piled party knife 
0.0%  : party
0.0%  : frogland
0.0%  : you've
0.0%  : zephyr
0.0%  : persuade
0.0%  : crow
0.0%  : requirements
0.0%  : impertinent
0.0%  : _be
0.0%  : critical
|


In [ ]:
model.save('./saved_models/aesop_no_dropout_100.h5')